MediaClassifier_a_PerClassAccuracy_v2-bigFeedSize
기존엔 64 x 64 를 네트워크에 feed 했지만, 256 x 256 을 feed 하고 싶다면, 나눠서 네트워크에 feed 함.
그렇지 않으면 OOM(out of memory) error 발생

* bsize = 100.
만약 너무 커서 OOM error 가 발생하면, bsize (out of memory) error 가 발생하면, bsize 를 줄여볼 것.
* 총 두 군데에서 bsize 를 사용함.
* 예) train_features = nets['relu5_4'].eval(feed_dict={img_placeholder: trainimg_tensor})

# Fine-tune CNN with pretrained VGG

In [11]:
# Import packs
import numpy as np
import os
import scipy.io
from scipy.misc import imread, imresize
import matplotlib.pyplot as plt
#import skimage.io
#import skimage.transform
import tensorflow as tf
%matplotlib inline  
cwd = os.getcwd()
print ("Package loaded")
print ("Current folder is %s" % (cwd) )

Package loaded
Current folder is /home/cgvmu/바탕화면/fookey1/PROJECT


# Load images and resize them to make a dataset

In [20]:
# Configure the locations of the images and reshaping sizes
# ------------------------------------------------------------------- #
#category_path = cwd + "/images/animal_dataset"
#category_path = cwd + "/images/car_dataset"
category_path = cwd + "/FOOD"
imgsize = [64, 64]      # The reshape size
use_gray = 0            # Grayscale
data_name = "data4vgg"  # Save name
valid_exts = [".jpg",".gif",".png",".tga", ".jpeg", ".bmp"]
# ------------------------------------------------------------------- #

    
main_categories = sorted(os.listdir(category_path))
main_nclass = len(main_categories)    

imgcnt = 0
sub_imgcnt = []
for i in range(0, main_nclass):
    sub_imgcnt.append(0)
    
sub_categories = []
sub_nclass = []
print("[%d] MAIN_CATEGORIES ARE IN \n %s\n" % (len(os.listdir(category_path)), category_path))
for i, mainCategory in zip(range(main_nclass), main_categories):
    sub_categories.append(sorted(os.listdir(category_path + "/" + mainCategory)))
    print("[%d] SUB_CATEGORIES ARE IN \n %s" % (len(os.listdir(category_path + "/" + mainCategory)), category_path + "/" + mainCategory))

## 계층적 파일 구조로 입력
for i in range(0, main_nclass):
    sublen = len(sub_categories[i])
    sub_nclass.append(sublen)
    for j in range(0, sub_nclass[i]):
        sub_category_path = category_path + "/" + main_categories[i] + "/" + sub_categories[i][j]
        for f in os.listdir(sub_category_path):
            fullpath = sub_category_path
            if os.path.splitext(f)[1].lower() not in valid_exts:
                continue
            fullpath = os.path.join(fullpath, f)
            imgcnt        = imgcnt + 1
            sub_imgcnt[i] = sub_imgcnt[i] + 1
        
print("[%d] PICTURES THERE" % imgcnt)
for i in range(0, main_nclass):
    print("[%d] PICTURES IN %s" % (sub_imgcnt[i], main_categories[i]))
# Grayscale
def rgb2gray(rgb):
    if len(rgb.shape) is 3:
        return np.dot(rgb[...,:3], [0.299, 0.587, 0.114]) #numpy.dot는 행렬에서의 요소별 곱
    else:
        print ("Current Image is GRAY!")
        return rgb
    
if use_gray:
    #totalimg   = np.ndarray((imgcnt, imgsize[0]*imgsize[1])) 
    
    subtotalimg = [np.ndarray(main_class, sub_imgcnt[i], imgsize[0] * imgsize[1]]
    for i in range(0, main_nclass):
        subtotalimg.append(np.ndarray((sub_imgcnt[i], imgsize[0]*imgsize[1])))
else:
    #totalimg   = np.ndarray((imgcnt, imgsize[0]*imgsize[1]*3))
    
    subtotalimg = []
    for i in range(0, main_nclass):
        subtotalimg.append(np.ndarray((sub_imgcnt[i], imgsize[0]*imgsize[1]*3)))
        
#numpy.ndarray는 n차원 배열을 만들어준다.(같은 종류의 데이터만 들어간다.)
#여기에서는 이미지의 w * h * clrnum(1 or 3, gray or RGB)을 데이터로 하는 총 이미지 개수만큼의 차원을 가진 배열을 만든다.
'''
totallabel = np.ndarray((imgcnt, main_nclass))
'''

sub_total_label = [];
for i in range(0, main_nclass):
    sub_total_label.append(np.ndarray((sub_imgcnt[i], sub_nclass[i])))
    

#각각의 이미지에 대해서 n개의 클래스에대한 가중치를 저장하는 공간

imgcnt     = 0
ignored_cnt = 0


sub_imgcnt = []
sub_ignored_cnt = []


#imgcnt_per_main_class = np.ndarray((main_nclass))


imgcnt_per_sub_class = [];
for i in range(0, main_nclass):
    imgcnt_per_sub_class.append(np.ndarray(sub_nclass[i]))
    sub_imgcnt.append(0)
    sub_ignored_cnt.append(0)



for i in range(0, main_nclass):
    #imgcnt_per_main_class[i] = 0
    for j in range(0, sub_nclass[i]):
        path = category_path + "/" + main_categories[i] + "/" + sub_categories[i][j]
        flist = os.listdir(path)
        
        
        imgcnt_per_sub_class[i][j] = 0
        
        for f in flist:
            if os.path.splitext(f)[1].lower() not in valid_exts:
                continue
            fullpath = os.path.join(path, f)
            currimg  = imread(fullpath)
            
            # Convert to grayscale  
            if use_gray:
                grayimg  = rgb2gray(currimg)
            else:
                grayimg  = currimg
                
            # (희경)에러가 나서 continue
            if (len(currimg.shape)==2 or currimg.shape[2]!=3):    
                #print("%s ignored" % f)
                ignored_cnt = ignored_cnt + 1
                sub_ignored_cnt[i] = sub_ignored_cnt[i] + 1
                continue 
            
            # Reshape -> numpy.reshape자주 나오는 함수: 배열의 차원을 바꿔서 할당해주는 함수
            # -1이 들어갈 경우 원본 배열을 다른 입력 차수 대로 할당하고 남은 차수로 할당되게 한다.
            graysmall = imresize(grayimg, [imgsize[0], imgsize[1]])/255.
            grayvec   = np.reshape(graysmall, (1, -1))

            #print(grayvec.shape)
            #print(totalimg[imgcnt, :].shape)
            # Save 
            '''
            totalimg[imgcnt, :]      = grayvec
            totallabel[imgcnt, :]    = np.eye(main_nclass, main_nclass)[i]
            imgcnt                   = imgcnt + 1
            imgcnt_per_main_class[i] = imgcnt_per_main_class[i] + 1 
            '''
            
            #'''
            subtotalimg[i][sub_imgcnt[i], :]     = grayvec
            sub_total_label[i][sub_imgcnt[i], :] = np.eye(sub_nclass[i], sub_nclass[i])[j]
            imgcnt                               = imgcnt + 1
            sub_imgcnt[i]                        = sub_imgcnt[i] + 1
            imgcnt_per_sub_class[i][j]           = imgcnt_per_sub_class[i][j] + 1  
            
            #'''


            

# Divide total data into training and test set
randidx    = np.random.randint(imgcnt, size=imgcnt)
trainidx   = randidx[0:int(70*imgcnt/100)]
#vaildation을 위해 새로 추가 - 원래 80:20이었는데 70:15:15로 변경
valididx   = randidx[int(70*imgcnt/100):int(85*imgcnt/100)]
#
testidx    = randidx[int(85*imgcnt/100):imgcnt]

main_ntrain = trainidx
main_nvalid = valididx
main_ntest  = testidx

subranidx = []
subtrainidx = []
subvalididx = []
subtestidx = []
for i in range(0, main_nclass):
    subranidx.append(np.random.randint(sub_imgcnt[i], size=sub_imgcnt[i]))
    subtrainidx.append(subranidx[i][0:int(70*sub_imgcnt[i]/100)])
    subvalididx.append(subranidx[i][int(70*sub_imgcnt[i]/100):int(85*sub_imgcnt[i]/100)])
    subtestidx.append(subranidx[i][int(85*sub_imgcnt[i]/100):sub_imgcnt[i]])

'''
trainimg   = totalimg[trainidx, :]
trainlabel = totallabel[trainidx, :]
#vaildation을 위해 새로 추가
validimg   = totalimg[valididx, :]
validlabel = totallabel[valididx, :]
#
testimg    = totalimg[testidx, :]
testlabel  = totallabel[testidx, :]

'''
subtrainimg = []
subtrainlabel = []
subvalidimg = []
subvalidlabel = []
subtestimg = []
subtestlabel = []
for i in range(0, main_nclass):
    subtrainimg.append(subtotalimg[i][subtrainidx[i], :]) 
    subtrainlabel.append(sub_total_label[i][subtrainidx[i], :])
    subvalidimg.append(subtotalimg[i][subvalididx[i], :]) 
    subvalidlabel.append(sub_total_label[i][subvalididx[i], :])
    subtestimg.append(subtotalimg[i][subtestidx[i], :]) 
    subtestlabel.append(sub_total_label[i][subtestidx[i], :])


main_ntrain     = trainimg.shape[0]
main_nclass     = trainlabel.shape[1]
#vaildation을 위해 새로 추가
main_nvalid     = validimg.shape[0];
main_dimvalid   = validlabel.shape[1];
#   
# 궁금한거 1 : testimg에서 가져와야 할 것 같은 dim이 왜 trainimg에서 왔을까?? 질문하기...
# 생각해보면 결국 클래스 수는 똑같으니까 뭘로 해도 문제는 안될듯
main_ntest      = testimg.shape[0]
main_dim        = trainimg.shape[1]



sub_ntrain = []
sub_nclass = []
sub_nvalid = []
sub_dimvalid = []
sub_ntest = []
sub_dim = []
for i in range(0, main_nclass):
    sub_ntrain.append(subtrainimg[i].shape[0])
    sub_nclass.append(subtrainlabel[i].shape[1])
    sub_nvalid.append(subvalidimg[i].shape[0])
    sub_dimvalid.append(subvalidlabel[i].shape[1])
    sub_ntest.append(subtestimg[i].shape[0])
    sub_dim.append(subtrainimg[i].shape[1])

'''

print ("Number of total images is %d (train: %d, valid: %d, test: %d), ignored: %d" 
       % (imgcnt, main_ntrain, main_nvalid, main_ntest, ignored_cnt)) 
print ("Shape of an image is (%d, %d, %d)" % (imgsize[0], imgsize[1], 3))
print ("Number of classes is %d:" % main_nclass)
for i, category in enumerate(main_categories):
    print (" [%d] %s (%d)" % (i, category, imgcnt_per_main_class[i]))
    

'''

for i in range(0, main_nclass):
    print ("Number of sub_total images is %d (train: %d, valid: %d, test: %d), ignored: %d" 
           % (sub_imgcnt[i] , sub_ntrain[i], sub_nvalid[i], sub_ntest[i], sub_ignored_cnt[i])) 
    print ("Shape of an sub_image is (%d, %d, %d)" % (imgsize[0], imgsize[1], 3))
    print ("Number of sub_classes is %d:" % sub_nclass[i])
    for j, category in enumerate(sub_categories[i]):
        print (" [%d] %s (%d)" % (j, category, imgcnt_per_sub_class[i][j]))

[4] MAIN_CATEGORIES ARE IN 
 /home/cgvmu/바탕화면/fookey1/PROJECT/FOOD

[5] SUB_CATEGORIES ARE IN 
 /home/cgvmu/바탕화면/fookey1/PROJECT/FOOD/국찌개
[6] SUB_CATEGORIES ARE IN 
 /home/cgvmu/바탕화면/fookey1/PROJECT/FOOD/반찬
[3] SUB_CATEGORIES ARE IN 
 /home/cgvmu/바탕화면/fookey1/PROJECT/FOOD/밥
[6] SUB_CATEGORIES ARE IN 
 /home/cgvmu/바탕화면/fookey1/PROJECT/FOOD/요리
[7053] PICTURES THERE
[1465] PICTURES IN 국찌개
[2673] PICTURES IN 반찬
[783] PICTURES IN 밥
[2132] PICTURES IN 요리
Number of sub_total images is 1465 (train: 1025, valid: 220, test: 220), ignored: 0
Shape of an sub_image is (64, 64, 3)
Number of sub_classes is 5:
 [0] 김치찌개 (280)
 [1] 된장찌개 (284)
 [2] 라면 (300)
 [3] 미역국 (301)
 [4] 콩나물국 (300)
Number of sub_total images is 2673 (train: 1871, valid: 401, test: 401), ignored: 0
Shape of an sub_image is (64, 64, 3)
Number of sub_classes is 6:
 [0] 마늘장아찌 (346)
 [1] 시금치 (353)
 [2] 계란 후라이 (261)
 [3] 김구이 (404)
 [4] 김치 (789)
 [5] 멸치볶음 (520)
Number of sub_total images is 783 (train: 548, valid: 1

# Define VGG network structure

In [21]:
#반복적으로 사용되는 함수들이 이곳에 존재함
def net(data_path, input_image):
    #VGGNet에 사용할 Layer를 지정함
    layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',
        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',
        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',
        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',
        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
    )
    data = scipy.io.loadmat(data_path)
    mean = data['normalization'][0][0][0]
    mean_pixel = np.mean(mean, axis=(0, 1))
    weights = data['layers'][0]
    net = {}
    current = input_image
    for i, name in enumerate(layers):
        kind = name[:4]
        if kind == 'conv':
            kernels, bias = weights[i][0][0][0][0]
            # matconvnet: weights are [width, height, in_channels, out_channels]
            # tensorflow: weights are [height, width, in_channels, out_channels]
            kernels = np.transpose(kernels, (1, 0, 2, 3))    # matlab형태의 데이터를 변형시켜 준다.
            bias = bias.reshape(-1)                          # 1차원 배열로 변경한다.
            current = _conv_layer(current, kernels, bias)    # conv layer 적용
        elif kind == 'relu':                                 # tf.nn은 tensorflow의 neural network 의미
            current = tf.nn.relu(current)                    # relu function적용 tensorflow 내장 함수임            
        elif kind == 'pool':        
            current = _pool_layer(current)                   # pooling 적용
        net[name] = current
    assert len(net) == len(layers)                           # feature map으 수와 layer의 수가 같은지판단: 정상 작동했다면 같은 개수일 것이다.

    return net, mean_pixel
def _conv_layer(input, weights, bias):
    conv = tf.nn.conv2d(input, tf.constant(weights), strides=(1, 1, 1, 1),
            padding='SAME')
    return tf.nn.bias_add(conv, bias)
def _pool_layer(input):
    return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),
            padding='SAME')
def preprocess(image, mean_pixel):
    return image - mean_pixel
def unprocess(image, mean_pixel):
    return image + mean_pixel
print ("VGG net ready")

VGG net ready


# Compute convoultional feature maps using VGG

In [41]:
# Preprocess
# training과 testing에 관한 데이터를 저장하는 다차원 배열을 할당 -- 추가로 validation할당
trainimg_tensor = np.ndarray((main_ntrain, imgsize[0], imgsize[1], 3))
testimg_tensor = np.ndarray((main_ntest, imgsize[0], imgsize[1], 3))
validimg_tensor = np.ndarray((main_nvalid, imgsize[0], imgsize[1], 3))#--valid 추가
# training과 testing에 관한 다차원 배열 (Tensor)에 초기 데이터를 집어넣음


for i in range(main_nclass):
    for j in range(sub_ntrain[i]):
        currimg = subtrainimg[i][j, :]
        currimg = np.reshape(currimg, [imgsize[0], imgsize[1], 3])
        trainimg_tensor[i, :, :, :] = currimg
print ("Shape of trainimg_tensor is %s" % (trainimg_tensor.shape,))    

    
for i in range(main_nclass):
    for j in range(sub_ntest[i]):
        currimg = subtestimg[i][j, :]
        currimg = np.reshape(currimg, [imgsize[0], imgsize[1], 3])
        testimg_tensor[i, :, :, :] = currimg 
print ("Shape of testimg_tensor is %s" % (testimg_tensor.shape,))


for i in range(main_nclass):#--valid 추가
    for j in range(sub_nvalid[i]):
        currimg = subvalidimg[i][j, :]
        currimg = np.reshape(currimg, [imgsize[0], imgsize[1], 3])
        validimg_tensor[i, :, :, :] = currimg 
print ("Shape of validimg_tensor is %s" % (validimg_tensor.shape,))
'''
sub_trainimg_tensor = []
sub_testimg_tensor = []
sub_validimg_tensor = []
for i in range(main_nclass):
    sub_trainimg_tensor.append(np.ndarray((sub_ntrain[i], imgsize[0], imgsize[1], 3)))
    sub_testimg_tensor.append(np.ndarray((sub_ntest[i], imgsize[0], imgsize[1], 3)))
    sub_validimg_tensor.append(np.ndarray((sub_nvalid[i], imgsize[0], imgsize[1], 3)))
    for j in range(sub_ntrain[i]):
        currimg = subtrainimg[i][j, :]
        currimg = np.reshape(currimg, [imgsize[0], imgsize[1], 3])
        sub_trainimg_tensor[i][j, :, :, :] = currimg 
    print ("Shape of sub_trainimg_tensor is %s" % (sub_trainimg_tensor[i].shape,))    

    for j in range(sub_ntest[i]):
        currimg = subtestimg[i][j, :]
        currimg = np.reshape(currimg, [imgsize[0], imgsize[1], 3])
        sub_testimg_tensor[i][j, :, :, :] = currimg 
    print ("Shape of sub_testimg_tensor is %s" % (sub_testimg_tensor[i].shape,))

    for j in range(sub_nvalid[i]):#--valid 추가
        currimg = subvalidimg[i][j, :]
        currimg = np.reshape(currimg, [imgsize[0], imgsize[1], 3])
        sub_validimg_tensor[i][j, :, :, :] = currimg 
    print ("Shape of sub_validimg_tensor is %s" % (sub_validimg_tensor[i].shape,))
'''
# batch 로 나누어 실행 - 나누어 처리하기위해서 추가된 코드.
bsize = 50
fwid = imgsize[0] / 16
fhei = imgsize[1] / 16
b_trainimg_tensor = np.ndarray((bsize, imgsize[0], imgsize[1], 3))
b_testimg_tensor = np.ndarray((bsize, imgsize[0], imgsize[1], 3))
b_validimg_tensor = np.ndarray((bsize, imgsize[0], imgsize[1], 3)) #--valid 추가
train_features = np.ndarray((main_ntrain, fwid, fhei, 512))
test_features = np.ndarray((main_ntest, fwid, fhei, 512))
valid_features = np.ndarray((main_nvalid, fwid, fhei, 512)) #--valid 추가

# Get conv features
# 각각의 이미지에 대해서 세션을 열고(with의 역할) training과 testing을 실시한다. 
VGG_PATH = cwd + "/data/mainCategory-vgg-verydeep-19.mat"
with tf.Graph().as_default(), tf.Session() as sess:
    with tf.device("/gpu:0"):
        img_placeholder = tf.placeholder(tf.float32
                                         , shape=(None, imgsize[0], imgsize[1], 3))
        nets, mean_pixel = net(VGG_PATH, img_placeholder)
        
        ##train_features = nets['relu5_4'].eval(feed_dict={img_placeholder: trainimg_tensor})
        ##test_features  = nets['relu5_4'].eval(feed_dict={img_placeholder: testimg_tensor})
        ##valid_features = nets['relu5_4'].eval(feed_dict={img_placeholder: vaildimg_tensor})
        
        #'''
        # 나눠서 해보자.
        num_batch = int(main_ntrain/bsize)+1
        for i in range(num_batch):
            start = i*bsize
            last = (i+1)*bsize
            if (last>main_ntrain): 
                last = main_ntrain
            b_trainimg_tensor = trainimg_tensor[start: last-1, :, :, :]
            b_train_features = nets['relu5_4'].eval(feed_dict={img_placeholder: b_trainimg_tensor})
            train_features[start: last-1, :, :, :] = b_train_features
        
        num_batch = int(main_ntest/bsize)+1
        for i in range(num_batch):
            start = i*bsize
            last = (i+1)*bsize
            if (last>main_ntest): 
                last = main_ntest
            b_testimg_tensor = testimg_tensor[start: last-1, :, :, :]
            b_test_features = nets['relu5_4'].eval(feed_dict={img_placeholder: b_testimg_tensor})
            test_features[start: last-1, :, :, :] = b_test_features
            
        num_batch = int(main_nvalid/bsize)+1#--valid 추가
        for i in range(num_batch):
            start = i*bsize
            last = (i+1)*bsize
            if (last>main_nvalid): 
                last = main_nvalid
            b_validimg_tensor = validimg_tensor[start: last-1, :, :, :]
            b_valid_features = nets['relu5_4'].eval(feed_dict={img_placeholder: b_validimg_tensor})
            valid_features[start: last-1, :, :, :] = b_valid_features
        #'''
    
print("Convolutional map extraction done")

Shape of trainimg_tensor is (4937, 64, 64, 3)
Shape of testimg_tensor is (1058, 64, 64, 3)
Shape of validimg_tensor is (1058, 64, 64, 3)
Convolutional map extraction done


# Our conv feature map looks like

In [42]:
print ("Shape of 'train_features' is %s" % (train_features.shape,))
print ("Shape of 'test_features' is %s" % (test_features.shape,))
print ("Shape of 'valid_features' is %s" % (valid_features.shape,))

Shape of 'train_features' is (4937, 4, 4, 512)
Shape of 'test_features' is (1058, 4, 4, 512)
Shape of 'valid_features' is (1058, 4, 4, 512)


# Vectorize 

In [43]:
# Vectorize
train_vectorized = np.ndarray((main_ntrain, fwid*fhei*512))
test_vectorized  = np.ndarray((main_ntest, fwid*fhei*512))
valid_vectorized  = np.ndarray((main_nvalid, fwid*fhei*512))
for i in range(main_ntrain):
    curr_feat = train_features[i, :, :, :]
    curr_feat_vec = np.reshape(curr_feat, (1, -1))
    train_vectorized[i, :] = curr_feat_vec
for i in range(main_ntest):
    curr_feat = test_features[i, :, :, :]
    curr_feat_vec = np.reshape(curr_feat, (1, -1))
    test_vectorized[i, :] = curr_feat_vec
for i in range(main_nvalid):#--valid 추가
    curr_feat = valid_features[i, :, :, :]
    curr_feat_vec = np.reshape(curr_feat, (1, -1))
    valid_vectorized[i, :] = curr_feat_vec
    
print ("Shape of 'train_vectorized' is %s" % (train_vectorized.shape,))
print ("Shape of 'test_vectorized' is %s" % (test_vectorized.shape,))

print ("Shape of 'valid_vectorized' is %s" % (valid_vectorized.shape,))#--valid 추가

Shape of 'train_vectorized' is (4937, 8192)
Shape of 'test_vectorized' is (1058, 8192)
Shape of 'valid_vectorized' is (1058, 8192)


# Define MLP for finetuning

In [44]:
# Parameters
learning_rate   = 0.0001    # learning rate
training_epochs = 100       # 트레이닝 횟수
batch_size      = 100       # 한번에 처리할 개수 - Out Of Memory(OOM)에러가 나면 줄여서 테스트 해 본다.
display_step    = 10        
# tf Graph input            # tensorflow.placeholder는 공간을 할당하는데 
                            # firstParam * secondParam의 크기로 할당이 된다. None이 들어오면 아무 값이나 할 사용할 수 있다.
x = tf.placeholder(tf.float32, [None, fwid*fhei*512])    # 입력값이 들어갈 공간
y = tf.placeholder(tf.float32, [None, main_nclass])           # 출력값이 들어갈 공간
keepratio = tf.placeholder(tf.float32)                   # drop out할 때 필요 - 여기에 저장해 놓고 나중에 drop out한 양 계산 
# Network
with tf.device("/gpu:0"):
    n_input  = main_dim
    n_output = main_nclass
    weights  = {
        'wd1': tf.Variable(tf.random_normal([fwid*fhei*512, 1024], stddev=0.1)),
        'wd2': tf.Variable(tf.random_normal([1024, n_output], stddev=0.1))
    }
    biases   = {
        'bd1': tf.Variable(tf.random_normal([1024], stddev=0.1)),
        'bd2': tf.Variable(tf.random_normal([n_output], stddev=0.1))
    }
    def conv_basic(_input, _w, _b, _keepratio):
        # Input
        _input_r = _input
        # Vectorize
        _dense1 = tf.reshape(_input_r, [-1, _w['wd1'].get_shape().as_list()[0]]) # input을 일렬로 펴기
        # Fc1
        _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1, _w['wd1']), _b['bd1']))
        _fc_dr1 = tf.nn.dropout(_fc1, _keepratio)
        # Fc2
        _out = tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2'])
        # Return everything
        out = {'input_r': _input_r, 'dense1': _dense1,
            'fc1': _fc1, 'fc_dr1': _fc_dr1, 'out': _out }
        return out
    # Functions! 
    # 여기에 있는 것들은 수행하는게 아니라 함수를 미리 선언해 놓은 부분이다. 아래에서 쓸 예정 
    _pred = conv_basic(x, weights, biases, keepratio)['out']
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred, labels=y))
    optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    _corr = tf.equal(tf.argmax(_pred,1), tf.argmax(y,1))            # 추측 pred와 정답 y가 같은지 비교하는 함수
    accr = tf.reduce_mean(tf.cast(_corr, tf.float32))               # 정답의 평균, 정확도 
    init = tf.global_variables_initializer()                            # 변수 초기화 - 다음에 트레이닝 할 때 필요하다.

print ("Network Ready to Go!")

Network Ready to Go!


# Optimize

In [45]:
# 이부분에서 validation을 통해서 끝내주는 부분이 있어야 할 것 같다. - 정확한 이해 없이는 진행이 되지 않을 것으로 보인다 - 교수님께 설명 다시 부탁 드려야 할듯
# Launch the graph
sess = tf.Session()
sess.run(init)

# Training cycle
for epoch in range(training_epochs):
#for epoch in range(20):
    avg_cost = 0.
    num_batch = int(main_ntrain/batch_size)+1      #
    # Loop over all batches
    for i in range(num_batch): 
        randidx  = np.random.randint(main_ntrain, size=batch_size)
        batch_xs = train_vectorized[randidx, :]
        batch_ys = trainlabel[randidx, :]                
        # Fit training using batch data
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys, keepratio:0.7})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keepratio:1.})/num_batch

    # Display logs per epoch step
    if epoch % display_step == 0 or epoch == training_epochs-1:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys, keepratio:1.})
        print (" Training accuracy: %.3f" % (train_acc))
        test_acc = sess.run(accr, feed_dict={x: test_vectorized, y: testlabel, keepratio:1.})
        print (" Test accuracy: %.3f" % (test_acc))
        valid_acc = sess.run(accr, feed_dict={x: valid_vectorized, y: validlabel, keepratio:1.})
        print (" Valid accuracy: %.3f" % (valid_acc))

        #test_corr = sess.run(_corr, feed_dict={x: test_vectorized, y: testlabel, keepratio:1.})

print ("Optimization Finished!")

Epoch: 000/100 cost: 1.993212478
 Training accuracy: 0.290
 Test accuracy: 0.294
 Valid accuracy: 0.296
Epoch: 010/100 cost: 1.304077144
 Training accuracy: 0.260
 Test accuracy: 0.301
 Valid accuracy: 0.297
Epoch: 020/100 cost: 1.297988052
 Training accuracy: 0.480
 Test accuracy: 0.370
 Valid accuracy: 0.377
Epoch: 030/100 cost: 1.307064106
 Training accuracy: 0.380
 Test accuracy: 0.369
 Valid accuracy: 0.377
Epoch: 040/100 cost: 1.303842182
 Training accuracy: 0.430
 Test accuracy: 0.369
 Valid accuracy: 0.377
Epoch: 050/100 cost: 1.301389501
 Training accuracy: 0.300
 Test accuracy: 0.369
 Valid accuracy: 0.377
Epoch: 060/100 cost: 1.305513086
 Training accuracy: 0.420
 Test accuracy: 0.373
 Valid accuracy: 0.380
Epoch: 070/100 cost: 1.305087490
 Training accuracy: 0.470
 Test accuracy: 0.373
 Valid accuracy: 0.380
Epoch: 080/100 cost: 1.301108062
 Training accuracy: 0.440
 Test accuracy: 0.369
 Valid accuracy: 0.377
Epoch: 090/100 cost: 1.281972201
 Training accuracy: 0.470
 Test

# +Per-Class Accuracy

In [ ]:
# y_label: one-hot coding. testlabel
# corr: 0 or 1. 
def perClassAccuracy(y_label, corr):
    # per-class accuracy 
    
    pc_accr = np.ndarray((main_nclass), dtype=float)  # # of class
    pc_cnt = np.ndarray((main_nclass))
    for i in range(main_nclass):
        pc_accr[i] = 0.
        pc_cnt[i] = 0
    
    # counting per-class accuracy 
    for i in range(y_label.shape[0]):
        y_ = y_label[i]
        idx = np.argmax(y_)
        pc_accr[idx] = pc_accr[idx] + corr[i]
        pc_cnt[idx] += 1
        

    print("*Per-class Accuracy")
    for i, category in enumerate(main_categories):
        #n_img = pc_cnt[i]
        pc_accr[i] = pc_accr[i] / pc_cnt[i]
        print(" %s (%d): %f" % (category, pc_cnt[i], pc_accr[i]))                

In [ ]:
test_accr = sess.run(accr, feed_dict={x:test_vectorized, y:testlabel, keepratio:1.})
print("*Test total accuracy: %.3f" % (test_accr))

test_corr = sess.run(_corr, feed_dict={x: test_vectorized, y: testlabel, keepratio:1.})
perClassAccuracy(testlabel, test_corr)

# +Per-Class Accuracy for New Test Data

In [ ]:

# --- 1) Load images and resize them to make a dataset ---
imgcnt = 0
newtestcategory_path = cwd + "/IMG"
newcategories = sorted(os.listdir(newtestcategory_path))
nnewclass = len(newcategories)
print("[%d] CATEGORIES ARE IN \n %s" % (len(os.listdir(newtestcategory_path)), newtestcategory_path))
for category in newcategories:
    for f in os.listdir(newtestcategory_path + "/" + category):
        fullpath = cwd + "/" + category
        if os.path.splitext(f)[1].lower() not in valid_exts:
            continue        
        fullpath = os.path.join(fullpath, f)
        imgcnt = imgcnt + 1
        
if use_gray:
    newtest_totalimg   = np.ndarray((imgcnt, imgsize[0]*imgsize[1]))
else:
    newtest_totalimg   = np.ndarray((imgcnt, imgsize[0]*imgsize[1]*3))
newtest_totallabel = np.ndarray((imgcnt, nnewclass))
imgcnt     = 0
ignored_cnt = 0
imgcnt_per_class = np.ndarray((nnewclass))
for i, relpath in zip(range(nnewclass), newcategories):
    path = newtestcategory_path + "/" + relpath
    flist = os.listdir(path)
    imgcnt_per_class[i] = 0
    for f in flist:
        if os.path.splitext(f)[1].lower() not in valid_exts:
            continue
        fullpath = os.path.join(path, f)
        currimg  = imread(fullpath)

        #print(f)
        # Convert to grayscale  
        if use_gray:
            grayimg  = rgb2gray(currimg)
        else:
            grayimg  = currimg
        
        # (희경)에러가 나서 continue
        if (len(currimg.shape)==2 or currimg.shape[2]!=3):    
            print("%s ignored" % f)
            ignored_cnt = ignored_cnt + 1
            continue 
                            
        # Reshape
        graysmall = imresize(grayimg, [imgsize[0], imgsize[1]])/255.
        grayvec   = np.reshape(graysmall, (1, -1))
        
        # Save 
        newtest_totalimg[imgcnt, :] = grayvec
        newtest_totallabel[imgcnt, :] = np.eye(nclass, nclass)[i]
        imgcnt    = imgcnt + 1
        imgcnt_per_class[i] = imgcnt_per_class[i] + 1
nnewtest = imgcnt

# 이 데이터는 모두 테스트 용이므로 따로 train, test data 를 나누지 않는다.
print ("Number of total images is %d, ignored: %d" 
       % (imgcnt, ignored_cnt)) 
print ("Shape of an image is (%d, %d, %d)" % (imgsize[0], imgsize[1], 3))
print ("Number of classes is %d:" % nnewclass)
for i, category in enumerate(newcategories):
    print (" [%d] %s (%d)" % (i, category, imgcnt_per_class[i]))

# --- 2) Tesorize, preprocess and vectorize them ---
newtest_totalimg_tensor = np.ndarray((nnewtest, imgsize[0], imgsize[1], 3))
for i in range(nnewtest):
    currimg = newtest_totalimg[i, :]
    currimg = np.reshape(currimg, [imgsize[0], imgsize[1], 3])
    newtest_totalimg_tensor[i, :, :, :] = currimg 
print ("Shape of newtestimg_tensor is %s" % (newtest_totalimg_tensor.shape,))

# 나눠서 실행
b_newtest_totalimg_tensor = np.ndarray((bsize, imgsize[0], imgsize[1], 3))
newtest_features = np.ndarray((nnewtest, fwid, fhei, 512))

# Get conv features
VGG_PATH = cwd + "/data/imagenet-vgg-verydeep-19.mat"
with tf.device("/gpu:0"):
    img_placeholder = tf.placeholder(tf.float32
                                         , shape=(None, imgsize[0], imgsize[1], 3))
    nets, mean_pixel = net(VGG_PATH, img_placeholder)
    
    # --- new test ---
    #newtest_features = sess.run(nets['relu5_4'], feed_dict={img_placeholder: newtest_totalimg_tensor})
    
    # 나눠서 해보자.
    bsize = 100
    num_batch = int(nnewtest/bsize) + 1
    for i in range(num_batch):
        start = i*bsize
        last = (i+1)*bsize
        if (last>nnewtest): 
            last = nnewtest
        b_newtest_totalimg_tensor = newtest_totalimg_tensor[start: last-1, :, :, :]
        b_newtest_features = sess.run(nets['relu5_4'], feed_dict={img_placeholder: b_newtest_totalimg_tensor})
        newtest_features[start: last-1, :, :, :] = b_newtest_features
    
print("Convolutional map extraction done")
print ("Shape of 'newtest_features' is %s" % (newtest_features.shape,))

# --- vectorize ---
newtest_vectorized = np.ndarray((nnewtest, fwid*fhei*512))
for i in range(nnewtest):
    curr_feat = newtest_features[i, :, :, :]
    curr_feat_vec = np.reshape(curr_feat, (1, -1))
    newtest_vectorized[i, :] = curr_feat_vec
    
print ("Shape of 'newtest_vectorized' is %s" % (newtest_vectorized.shape,))


In [ ]:

# Data| 의 test image accuracy
test_accr = sess.run(accr, feed_dict={x:test_vectorized, y:testlabel, keepratio:1.})
#print("[Real Artwork]")
print("[%s]" % (category_path))
print("*total test accuracy: %.3f" % (test_accr))

test_corr = sess.run(_corr, feed_dict={x: test_vectorized, y: testlabel, keepratio:1.})
perClassAccuracy(testlabel, test_corr)

In [ ]:

# 3) Estimate accuracy 
newtest_acc = sess.run(accr, feed_dict={x: newtest_vectorized, y: newtest_totallabel, keepratio:1.})
#print("[Synthesized Image]")
print("[%s]" % (newtestcategory_path))
print("*New test total accuracy: %.3f" % (newtest_acc))

newtest_corr = sess.run(_corr, feed_dict={x: newtest_vectorized, y: newtest_totallabel, keepratio:1.})
perClassAccuracy(newtest_totallabel, newtest_corr)


# +Visualize true and false predicted images

In [ ]:

import math
def plot_images(images, cls_true, cls_pred=None):
    nimg = len(images)
    # Create figure with nxn sub-plots.
    num_grid = math.ceil(math.sqrt(nimg))
    print num_grid
    if num_grid > 1.0:
        fig, axes = plt.subplots(int(num_grid), int(num_grid))
        fig.subplots_adjust(hspace=0.9, wspace=0.3)

        for i, ax in enumerate(axes.flat):
            if (i>=nimg):
                break

            # Plot image.
            ax.imshow(images[i])

            trueid = np.argmax(cls_true[i])  # 정답 label -> true id

            # Show true and predicted classes
            if cls_pred is None:
                xlabel = "True: %d (%s)" %(trueid, categories[trueid])
            else:
                predid = np.argmax(cls_pred[i])
                xlabel = "Predicted: %d (%s) \n True: %d (%s)" %(predid, categories[predid], trueid, categories[trueid])

            # Show the classes as the label on the x-axis.
            ax.set_xlabel(xlabel.decode('utf-8'))

            # Remove ticks from the plot.
            ax.set_xticks([])
            ax.set_yticks([])

        # Ensure the plot is shown correctly with multiple plots
        # in a single Notebook cell.
        plt.show()

In [ ]:
def ShowPredImgs(_y_label, _pred, _icorr, _img_tensor, _n=3):
    len_data = len(_y_label)
    # 1. true, false list array 할당
    tlist = np.ndarray((nclass, len_data))  # true predicted list
    flist = np.ndarray((nclass, len_data))  # false predicted list
    tcnt = np.ndarray((nclass))
    fcnt = np.ndarray((nclass))
        
    # 2. 1 채우기
    for i in range(nclass):
        tcnt[i] = 0   
        fcnt[i] = 0
    # 각 클래스마다 true predicted 이미지 개수 세기
    for i, crr in enumerate(_icorr):
        classid = np.argmax(_y_label[i])  # 정답 label -> class id
        if (crr):    # true predicted 라면
            cnt = int(tcnt[classid])       # classid 의 true counting 증가
            tcnt[classid] += 1
            tlist[classid][cnt] = i   # true predicted index 추가
        else:
            cnt = int(fcnt[classid])
            fcnt[classid] += 1
            flist[classid][cnt] = i
            
    for k, category in enumerate(categories):
        print("Category: %s" % (category))
        tnew_n = _n
        fnew_n = _n
        
        # 3. rand idx (중복 선택 허용)
        #trandidx = np.random.randint(tcnt[k], size=_n)   
        #frandidx = np.random.randint(fcnt[k], size=_n)
        # 중복 X        
        if (tcnt[k]==0):
            tnew_n = 0
        elif (tcnt[k]<_n):   # 요청하는 개수보다 전체 개수가 더 적으면
            trandidx = np.random.choice(int(tcnt[k]), int(tcnt[k]), replace=False)
            tnew_n = int(tcnt[k])
        else:
            trandidx = np.random.choice(int(tcnt[k]), _n, replace=False)
        
        if (fcnt[k]==0):
            fnew_n = 0
        elif (fcnt[k]<_n):
            frandidx = np.random.choice(int(fcnt[k]), int(fcnt[k]), replace=False)
            fnew_n = int(fcnt[k])
        else:
            frandidx = np.random.choice(int(fcnt[k]), _n, replace=False)
        #print(trandidx)
        #print(frandidx)
    
        # 4. n 개 true, false 그리기
        # 1) true predicted image 그리기
        print(" True predicted images/total %s category: %d / %d" % (category.decode('utf-8'), tcnt[k], tcnt[k]+fcnt[k]))
        for i in range(tnew_n):
            #print("trandidx: %d" % trandidx[i])
            curridx = int(tlist[k][trandidx[i]])
            plt.imshow(_img_tensor[curridx])
            
            pre_label = np.argmax(_pred[curridx])
            print("Predicted: %d (%s) / True: %d (%s)" % 
                      (pre_label, categories[pre_label].decode('utf-8'),
                      k, category.decode('utf-8')))
            plt.title("Predicted: %d (%s) / True: %d (%s)" % 
                      (pre_label, categories[pre_label].decode('utf-8'),
                      k, category.decode('utf-8')))
            plt.show()
        
        # 2) false predicted image 그리기
        print(" False predicted images/total %s category: %d / %d" % (category.decode('utf-8'), fcnt[k], tcnt[k]+fcnt[k]))
        for i in range(fnew_n):
            #print("frandidx: %d" % frandidx[i])
            curridx = int(flist[k][frandidx[i]])
            plt.imshow(_img_tensor[curridx])
            
            pre_label = np.argmax(_pred[curridx])
            print("Predicted: %d (%s) / True: %d (%s)" % 
                      (pre_label, categories[pre_label].decode('utf-8'),
                      k, category.decode('utf-8')))
            plt.title("Predicted: %d (%s) / True: %d (%s)" % 
                      (pre_label, categories[pre_label].decode('utf-8'),
                      k, category.decode('utf-8')))
            plt.show()           
    
        # 5. pred, true 출력
        
                
        # 1) true predicted image 그리기
        print(" True predicted images/total %s category: %d / %d" % (category, tcnt[k], tcnt[k]+fcnt[k]))
        i_img = np.ndarray((tnew_n, _img_tensor.shape[1], _img_tensor.shape[2], _img_tensor.shape[3]))
        i_true = np.ndarray((tnew_n, nclass))
        i_pred = np.ndarray((tnew_n, nclass))
        for i in range(tnew_n):
            curridx = int(tlist[k][trandidx[i]])
            i_img[i] = _img_tensor[curridx]
            i_true[i] = _y_label[curridx]
            i_pred[i] = _pred[curridx]
        plot_images(i_img, i_true, i_pred)
                
        # 2) false predicted image 그리기
        print(" False predicted images/total %s category: %d / %d" % (category, fcnt[k], tcnt[k]+fcnt[k]))
        i_img = np.ndarray((fnew_n, _img_tensor.shape[1], _img_tensor.shape[2], _img_tensor.shape[3]))
        i_true = np.ndarray((fnew_n, nclass))
        i_pred = np.ndarray((fnew_n, nclass))
        for i in range(fnew_n):
            curridx = int(flist[k][frandidx[i]])
            i_img[i] = _img_tensor[curridx]
            i_true[i] = _y_label[curridx]
            i_pred[i] = _pred[curridx]
        plot_images(i_img, i_true, i_pred)
        
        

In [ ]:
print ("[%s]" % (category_path))
my_pred = sess.run(_pred, feed_dict={x: test_vectorized, y:testlabel, keepratio:1.})
my_corr = sess.run(_corr, feed_dict={x: test_vectorized, y:testlabel, keepratio:1.})
ShowPredImgs(testlabel, my_pred, my_corr, testimg_tensor, 3)


print("[%s]" % (newtestcategory_path))
my_pred = sess.run(_pred, feed_dict={x: newtest_vectorized, y:newtest_totallabel, keepratio:1.})
my_corr = sess.run(_corr, feed_dict={x: newtest_vectorized, y:newtest_totallabel, keepratio:1.})
ShowPredImgs(newtest_totallabel, my_pred, my_corr, newtest_totalimg_tensor, 3)
